In [0]:
import pandas as pd
import numpy as np
import chess
import re
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn import utils
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import sklearn.metrics

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data = pd.read_csv('/content/gdrive/My Drive/cse519_project/new_final_position_pred.csv')
data = data[["Legal_Moves", "Piece_Num","DRAW",'Pred']]
data = pd.DataFrame(data).fillna(0)

In [4]:
for i in range(0, len(data)):
  if data['Pred'][i] < 100:
    data['Pred'][i] = 1
  elif data['Pred'][i] >= 100 and data['Pred'][i]<150:
    data['Pred'][i] = 2
  elif data['Pred'][i] >= 150 and data['Pred'][i]<200:
    data['Pred'][i] = 3
  elif data['Pred'][i] >= 200:
    data['Pred'][i] = 4

for i in range(0, len(data)):
  data['Legal_Moves'][i] = data['Legal_Moves'][i]/10

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/

In [0]:
sns.set()
plt.figure(figsize=(5,5))
sns.distplot(data['Pred'], color ="#4e4f63")
plt.show()

In [0]:
sns.set()
plt.figure(figsize=(5,5))
sns.countplot(data['Pred'], palette="bone_r")
plt.show()

In [0]:
plt.figure(figsize=(6,6))
corr_matrix = data.corr()
mask = np.zeros_like(corr_matrix)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
  p2 = sns.heatmap(corr_matrix, mask=mask, vmax=0.3, square=True, cmap="bone_r",annot=True)

In [0]:
sns.set()
plt.figure(figsize=(6,6))
sns.regplot(x="Piece_Num", y="Pred", data=data, fit_reg=False, scatter_kws={"color":"#83919f","alpha":0.1,"s":5})
plt.show()

In [0]:
sns.set()
plt.figure(figsize=(10,5))
sns.countplot(data['Legal_Moves'], palette="bone_r")
plt.show()

In [0]:
sns.set()
plt.figure(figsize=(10,5))
sns.boxplot(x="Legal_Moves", y="Pred", data=data, palette="bone")
plt.ylim((0,400))
plt.show()

In [0]:
sns.set()
plt.figure(figsize=(6,6))
sns.stripplot(x='DRAW',y='Pred',hue='DRAW',jitter=True,dodge=True,size=20,marker='D',edgecolor='gray',alpha=.25,palette="bone",data=data)
plt.legend(loc=10)
plt.show()

**分割数据集**

In [7]:
features, target = data.ix[:,:3], data.ix[:,3]
train_data, test_data, train_target, test_target = train_test_split(features, target, test_size=0.2,random_state=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


In [8]:
model = LinearRegression()
model.fit(train_data, train_target)
pred = model.predict(test_data)
acc = model.score(test_data, test_target)
print('The accuracy is', acc*100,'%')

The accuracy is 86.0288188342371 %


In [9]:
print(sklearn.metrics.mean_squared_error(test_target,pred))
print(sklearn.metrics.r2_score(test_target,pred))

0.1316498433320195
0.860288188342371


In [10]:
nn = KNeighborsClassifier()
nn.fit(train_data, train_target)
pred2 = nn.predict(test_data)
acc2 = nn.score(test_data, test_target)
print('The accuracy is', acc2*100,'%')

The accuracy is 94.42027923040581 %


In [11]:
print(sklearn.metrics.mean_squared_error(test_target,pred2))
print(sklearn.metrics.r2_score(test_target,pred2))

0.055797207695941875
0.9407858849252346


In [12]:
 xgb_model = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=260,
 max_depth=5,
 min_child_weight=2,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= "binary:logistic",
 nthread=4,
 scale_pos_weight=1,
 seed=27)
xgb_model.fit(train_data, train_target)
pred1 = xgb_model.predict(test_data)
acc1 = xgb_model.score(test_data, test_target)
print('The accuracy is', acc1*100,'%')

The accuracy is 94.69397064333774 %


In [13]:
print(sklearn.metrics.mean_squared_error(test_target,pred1))
print(sklearn.metrics.r2_score(test_target,pred1))

0.05306029356662268
0.9436904021026246


In [14]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(objective='multiclass', random_state=5)
lgbm.fit(train_data, train_target)
pred3= lgbm.predict(test_data)
acc3 = lgbm.score(test_data, test_target)
print('The accuracy is', acc3*100,'%')

The accuracy is 94.69885798999724 %


In [15]:
print(sklearn.metrics.mean_squared_error(test_target,pred3))
print(sklearn.metrics.r2_score(test_target,pred3))

0.0530114201000277
0.9437422684807922


In [16]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
rfc.fit(train_data, train_target)
pred4= rfc.predict(test_data)
acc4 = rfc.score(test_data, test_target)
print('The accuracy is', acc4*100,'%') 

The accuracy is 81.16090774318623 %


In [19]:
print(sklearn.metrics.mean_squared_error(test_target,pred4))
print(sklearn.metrics.r2_score(test_target,pred4))

0.18873303683430265
0.7997093363091512
